#  Final project: the "1C" competition
This challenge serves as final project for the "How to win a data science competition" Coursera course.

In this competition you will work with a challenging time-series dataset consisting of daily sales data, kindly provided by one of the largest Russian software firms - 1C Company. 

We are asking you to predict total sales for every product and store in the next month. By solving this competition you will be able to apply and enhance your data science skills.

## File descriptions
* sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
* test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
* sample_submission.csv - a sample submission file in the correct format.
* items.csv - supplemental information about the items/products.
* item_categories.csv  - supplemental information about the items categories.
* shops.csv- supplemental information about the shops.

## Data fields
* ID - an Id that represents a (Shop, Item) tuple within the test set
* shop_id - unique identifier of a shop
* item_id - unique identifier of a product
* item_category_id - unique identifier of item category
* item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
* item_price - current price of an item
* date - date in format dd/mm/yyyy
* date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
* item_name - name of item
* shop_name - name of shop
* item_category_name - name of item category

# **Exploratory Data Analysis**
### Data Preprocessing & Feature Generation

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [36]:
DATA_FOLDER = "./Data"
#train data
transactions    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv'))
#supplement data
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv')) #item id and its category id
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv')) #item category name
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv')) #shop name
test            = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv')) #test data
sample_submission = pd.read_csv(os.path.join(DATA_FOLDER, 'sample_submission.csv')) #sample_submission

In [3]:
print("Shape of transactions: "+ str(transactions.shape))
transactions.head()

Shape of transactions: (2935849, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


### Recap from week1 data processing

In [4]:
# Make some useful features

# Remove dates
newcols = transactions["date"].str.split(".", expand=True)
ls = ["day", "month", "year"]
for i in range(len(ls)):
    transactions.insert(i, ls[i], newcols[i])
transactions.pop("date")

# Add item category for each item
transactions['item_category_id'] = [items['item_category_id'].values[i] for i in transactions['item_id'].values]

# Add revenues
transactions["revenue"] = transactions["item_price"]*transactions["item_cnt_day"]

In [5]:
transactions.head()

,day,month,year,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,revenue
0,02,01,2013,0,59,22154,999.00,1.0,37,999.00
1,03,01,2013,0,25,2552,899.00,1.0,58,899.00
2,05,01,2013,0,25,2552,899.00,-1.0,58,-899.00
3,06,01,2013,0,25,2554,1709.05,1.0,58,1709.05
4,15,01,2013,0,25,2555,1099.00,1.0,56,1099.00


In [6]:
len(transactions["item_id"].unique())

21807

**1. Add the shop total revenue which each transaction belong to and normalize it.**

In [7]:
sum_revenue = transactions.groupby(by="shop_id")["revenue"].sum()
transactions["shop_sum_revenue"] = transactions["shop_id"].map(sum_revenue)

In [8]:
transactions.head()

,day,month,year,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,revenue,shop_sum_revenue
0,02,01,2013,0,59,22154,999.00,1.0,37,999.00,4.522658e+07
1,03,01,2013,0,25,2552,899.00,1.0,58,899.00,2.164806e+08
2,05,01,2013,0,25,2552,899.00,-1.0,58,-899.00,2.164806e+08
3,06,01,2013,0,25,2554,1709.05,1.0,58,1709.05,2.164806e+08
4,15,01,2013,0,25,2555,1099.00,1.0,56,1099.00,2.164806e+08


In [9]:
scaler = MinMaxScaler()
scaler.fit(transactions["shop_sum_revenue"].values.reshape(-1, 1))
transactions["shop_sum_revenue"] = scaler.transform(transactions["shop_sum_revenue"].values.reshape(-1, 1))

In [10]:
transactions.head()

,day,month,year,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,revenue,shop_sum_revenue
0,02,01,2013,0,59,22154,999.00,1.0,37,999.00,0.190977
1,03,01,2013,0,25,2552,899.00,1.0,58,899.00,0.920216
2,05,01,2013,0,25,2552,899.00,-1.0,58,-899.00,0.920216
3,06,01,2013,0,25,2554,1709.05,1.0,58,1709.05,0.920216
4,15,01,2013,0,25,2555,1099.00,1.0,56,1099.00,0.920216


**2. Add the item_category total revenue which each transaction belong to and normalize it.**

In [11]:
sum_revenue = transactions.groupby(by="item_category_id")["revenue"].sum()
transactions["category_sum_revenue"] = transactions["item_category_id"].map(sum_revenue)
scaler.fit(transactions["category_sum_revenue"].values.reshape(-1, 1))
transactions["category_sum_revenue"] = scaler.transform(transactions["category_sum_revenue"].values.reshape(-1, 1))

shop_id_with_max_revenue =  transactions.groupby(by='shop_id')['revenue'].sum().idxmax() 
category_id_with_max_revenue =  transactions.groupby(by='item_category_id')['revenue'].sum().idxmax() 
print("SHOP ID with Max revenue: ",shop_id_with_max_revenue)
print("Category ID with Max revenue: ",category_id_with_max_revenue)

SHOP ID with Max revenue:  31
Category ID with Max revenue:  19


**3. Calculate the item's mean price and whether its price stays constant (1) or price fluctuations (0) for all period of time.**

In [12]:
grps = transactions.sort_values(by='item_id').groupby(by='item_id')
prices = grps['item_price'].mean()
transactions["mean_item_price"] = transactions.sort_values(by='item_id')['item_id'].map(prices)

transactions["price_count"] = np.nan
indexes1 = transactions[(transactions["mean_item_price"]==transactions["item_price"])].index
transactions.loc[indexes1,"price_count"] = transactions.loc[indexes1]["mean_item_price"].apply(lambda x: 1)
indexes2 = transactions[(transactions["mean_item_price"]!=transactions["item_price"])].index
transactions.loc[indexes2,"price_count"] = transactions.loc[indexes2]["mean_item_price"].apply(lambda x: 0)

In [13]:
# Mean Encoding with K-Fold regularization for item_category_id
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = False)
transactions['category_price_count_enc'] = np.nan

for tr_ind, val_ind in kf.split(transactions):
    X_tr, X_val = transactions.iloc[tr_ind], transactions.iloc[val_ind]
    means = X_val['item_category_id'].map(X_tr.groupby('item_category_id').price_count.mean())
    transactions.loc[val_ind, 'category_price_count_enc'] = means
    
mean = transactions["price_count"].mean()
transactions['category_price_count_enc'].fillna(mean, inplace = True)


# Calculate correlation coefficients
encoded_feature = transactions['category_price_count_enc'].values
corr = np.corrcoef(transactions['price_count'].values, encoded_feature)[0][1]
print(corr)

0.44491004194429146


In [14]:
# Mean Encoding with K-Fold regularization for shop_id
transactions['shop_price_count_enc'] = np.nan

for tr_ind, val_ind in kf.split(transactions):
    X_tr, X_val = transactions.iloc[tr_ind], transactions.iloc[val_ind]
    means = X_val['shop_id'].map(X_tr.groupby('shop_id').price_count.mean())
    transactions.loc[val_ind, 'shop_price_count_enc'] = means
    
mean = transactions["price_count"].mean()
transactions['shop_price_count_enc'].fillna(mean, inplace = True)


# Calculate correlation coefficients
encoded_feature = transactions['shop_price_count_enc'].values
corr = np.corrcoef(transactions['price_count'].values, encoded_feature)[0][1]
print(corr)

0.3685836030657191


In [15]:
transactions.head(10)

,day,month,year,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,revenue,shop_sum_revenue,category_sum_revenue,mean_item_price,price_count,category_price_count_enc,shop_price_count_enc
0,02,01,2013,0,59,22154,999.00,1.0,37,999.00,0.190977,0.219582,702.932203,0.0,0.006592,0.005472
1,03,01,2013,0,25,2552,899.00,1.0,58,899.00,0.920216,0.056344,937.888889,0.0,0.164006,0.024701
2,05,01,2013,0,25,2552,899.00,-1.0,58,-899.00,0.920216,0.056344,937.888889,0.0,0.164006,0.024701
3,06,01,2013,0,25,2554,1709.05,1.0,58,1709.05,0.920216,0.056344,1709.050000,1.0,0.164006,0.024701
4,15,01,2013,0,25,2555,1099.00,1.0,56,1099.00,0.920216,0.023055,1123.101786,0.0,0.134168,0.024701
5,10,01,2013,0,25,2564,349.00,1.0,59,349.00,0.920216,0.006858,340.152174,0.0,0.130000,0.024701
6,02,01,2013,0,25,2565,549.00,1.0,56,549.00,0.920216,0.023055,549.000000,1.0,0.134168,0.024701
7,04,01,2013,0,25,2572,239.00,1.0,55,239.00,0.920216,0.242743,295.157895,0.0,0.009279,0.024701
8,11,01,2013,0,25,2572,299.00,1.0,55,299.00,0.920216,0.242743,295.157895,0.0,0.009279,0.024701
9,03,01,2013,0,25,2573,299.00,3.0,55,897.00,0.920216,0.242743,254.155556,0.0,0.009279,0.024701


**4. Calculate the total item sold and its standard deviation**

In [16]:
total_num_items_sold = transactions.groupby(by="item_category_id")["item_cnt_day"].sum()
transactions["total_num_items_sold_std"] = transactions["item_category_id"].map(total_num_items_sold)
total_num_items_sold_std = np.std(total_num_items_sold, ddof=1)
print(total_num_items_sold_std)

scaler.fit(transactions["total_num_items_sold_std"].values.reshape(-1, 1))
transactions["total_num_items_sold_std"] = scaler.transform(transactions["total_num_items_sold_std"].values.reshape(-1, 1))

99846.02744394378


# **Feature Generation for training set**

In [17]:
transactions.head()

,day,month,year,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,revenue,shop_sum_revenue,category_sum_revenue,mean_item_price,price_count,category_price_count_enc,shop_price_count_enc,total_num_items_sold_std
0,02,01,2013,0,59,22154,999.00,1.0,37,999.00,0.190977,0.219582,702.932203,0.0,0.006592,0.005472,0.320550
1,03,01,2013,0,25,2552,899.00,1.0,58,899.00,0.920216,0.056344,937.888889,0.0,0.164006,0.024701,0.021493
2,05,01,2013,0,25,2552,899.00,-1.0,58,-899.00,0.920216,0.056344,937.888889,0.0,0.164006,0.024701,0.021493
3,06,01,2013,0,25,2554,1709.05,1.0,58,1709.05,0.920216,0.056344,1709.050000,1.0,0.164006,0.024701,0.021493
4,15,01,2013,0,25,2555,1099.00,1.0,56,1099.00,0.920216,0.023055,1123.101786,0.0,0.134168,0.024701,0.013056


In [18]:
# Generate item_cnt_month (y_train)
import warnings
warnings.filterwarnings("ignore")

new_transactions = pd.DataFrame()
for i in range(len(transactions["date_block_num"].unique())):
    # get monthly data
    each_month_data = transactions[transactions["date_block_num"]==i]
    # calculate each item count per month
    for i in each_month_data["shop_id"].unique():
        a = each_month_data.groupby("shop_id").get_group(i)
        each_month_data.loc[a.index, "item_cnt_month"] = a.groupby("item_id")["item_cnt_day"].transform('sum')
    # make new data
    new_transactions = new_transactions.append(each_month_data)

In [19]:
new_trans = new_transactions.drop(["day", "year", "item_cnt_day", "revenue"], axis=1)
new_trans = new_trans.drop_duplicates()

In [20]:
new_trans.head()

,month,date_block_num,shop_id,item_id,item_price,item_category_id,shop_sum_revenue,category_sum_revenue,mean_item_price,price_count,category_price_count_enc,shop_price_count_enc,total_num_items_sold_std,item_cnt_month
0,01,0,59,22154,999.00,37,0.190977,0.219582,702.932203,0.0,0.006592,0.005472,0.320550,1.0
1,01,0,25,2552,899.00,58,0.920216,0.056344,937.888889,0.0,0.164006,0.024701,0.021493,0.0
3,01,0,25,2554,1709.05,58,0.920216,0.056344,1709.050000,1.0,0.164006,0.024701,0.021493,1.0
4,01,0,25,2555,1099.00,56,0.920216,0.023055,1123.101786,0.0,0.134168,0.024701,0.013056,1.0
5,01,0,25,2564,349.00,59,0.920216,0.006858,340.152174,0.0,0.130000,0.024701,0.010423,1.0


# Train/Test Split
For a sake of the programming assignment, let's artificially split the data into train and test. We will treat last month data as the test set.

In [21]:
dates = new_trans['date_block_num']

last_block = dates.max()
print('Test `date_block_num` is %d' % last_block)

Test `date_block_num` is 33


In [22]:
dates_train = dates[dates <  last_block]
dates_test  = dates[dates == last_block]

to_drop_cols = ['month', 'date_block_num', 'item_price', 'price_count', 'item_cnt_month']

X_train = new_trans.loc[dates < last_block].drop(to_drop_cols, axis=1)
X_test = new_trans.loc[dates == last_block].drop(to_drop_cols, axis=1)
y_train = new_trans.loc[dates < last_block, "item_cnt_month"].values
y_test = new_trans.loc[dates == last_block, "item_cnt_month"].values

In [33]:
import xgboost as xgb

reg = xgb.XGBRegressor(n_estimators=100, max_depth=12, 
                       learning_rate=0.1, subsample=0.8, 
                       min_child_weight=300, colsample_bytree=1, 
                       eta=0.3, eval_metric='rmse')

reg.fit(X_train, y_train)
pred_cnt = reg.predict(X_test)

In [34]:
from sklearn.metrics import r2_score
r2_score(y_test, pred_cnt)

0.17670452262991831